# Python implementation of efficient edge recording

In [15]:
class Segment(object):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
class EdgeBuffer(object):
    def __init__(self):
        # Our linked lists managers
        self.head = [] # i-th value records the start of data for parent i.
        self.tail = []
        self.next = []
        # Our data
        self.left = []
        self.right = []
        self.child = []
        # helpers
        self.collected_children = dict()
        self.children = []
        
    def _clear():
        """
        Free up memory
        """
        self.head = [] # i-th value records the start of data for parent i.
        self.tail = []
        self.next = []
        self.left = []
        self.right = []
        self.child = []
        self.collected_children = dict()
        self.children = []
        
    def _buffer_edge(self, left, right, parent, child):
        if parent >= len(self.head):
            self.head.extend([-1]*(parent-len(self.head)-1))
            self.tail.extend([-1]*(parent-len(self.head)-1))
        if self.head[parent] == -1:
            self._insert_new_parent(left, right, parent, child)
        else:
            self._extend_parent(left, right, parent, child)
            
    def _insert_new_parent(self, left, right, parent, child):
        self.left.push(left)
        self.right.push(right)
        self.child.push(child)
        self.head[parent] = len(self.left) - 1
        self.tail[parent] = self.head[parent]
        self.next.push(-1)
    
    def _extend_parent(self, left, right, parent, child):
        self.left.push(left)
        self.right.push(right)
        self.child.push(child)
        temp = self.tail[parent]
        self.head[parent] = len(self.left) - 1
        self.next[temp] = len(self.left) - 1
        self.next.push(-1)
        
    # Can skip right to _buffer_edge w/modular simplification API
    def record_edge(self, left, right, parent, child):
        self.collected_children[parent][child].append(Segment(left, right))
    
    # Not necessary w/modular simplification API
    def init_recording_offspring(self, parents, children):
        self.children = sorted([i for i in set([j for j in children])])
        self.collected_children = dict()
        for p in parents:
            self.collected_children[p] = {c: [] for c in self.children}
    
    # Not necessary w/modular simplification API
    def finalize_recording_offspring(self):
        for p, edges in self.collected_children.items():
            for c in self.children:
                for e in self.collected_children[c]:
                    self._buffer_edge(e.left, e.right, p, c)

    # This implementation suffices for non-overlapping
    # generations.
    # Preconditions:
    # * for all i, self.head[i] are recorded in birth
    #   order, past to present, such that node time
    #   time[i] < time[i-1]
    # Notes:
    # * In C, this is done more efficiently by
    #   appending edge tables rows and then
    #   doing a "left rotation" of each column
    #   when done.
    def prep_for_simplification(self, tables):
        left = []
        right = []
        parent = []
        child = []
        
        # Go backwards in time through parents
        for i, h in enumerate(self.head[::-1]):
            if h != -1: # Skip NULL values (nodes that did not give birth)
                # convert reverse index to parent id
                parent = len(self.head) - i - 1
                left.append(self.left[h])
                right.append(self.right[h])
                parent.append(parent)
                child.append(self.child[h])
                
                next = self.next[h]
                while next != -1:
                    left.append(self.left[next])
                    right.append(self.right[next])
                    parent.append(parent)
                    child.append(self.child[next])
                    next = self.next[next]
                    
        # Append ancient edges
        left.extend(tables.edges.left.tolist())
        right.extend(tables.edges.right.tolist())
        parent.extend(tables.edges.parent.tolist())
        child.extend(tables.edges.child.tolist())
        
        # Set columns
        tables.edges.set_columns(left=left, right=right, parent=parent, child=child)
        
        # Free memory prior to simplifying
        self._clear()
        
        

In [17]:
# Haploid sim w/exactly one crossover per offspring
def overlapping_generations(popsize, nsteps, pdeath=1.0):
    assert pdeath == 1.0, "The buffer is not generalized to overlapping generations yet"
    tables = tskit.TableCollection(10)
    raise NotImplemented()